In [1]:
# import necessary libraries
import os
import json
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import create_engine
from geojson import Feature, FeatureCollection, Point
import csv
import pandas as pd
import pprint

#Creating engine object to the Depression Database
engine = create_engine('sqlite:///depression.db')
#Creating FileName variables pre CSV
npao_file_name = 'cdc_npao.csv'
depression_file_name = 'depression_data.csv'
alcohol_file_name = 'alcohol_data.csv'
income_file_name = 'income_data.csv'
poverty_file_name = 'poverty_data.csv'

#Reading file contents into respective DataFrame
npao_df = pd.read_csv(npao_file_name)
depression_df = pd.read_csv(depression_file_name)
depression_df = depression_df.filter(["STATE NAME", "Yes%"])
depression_df.rename(columns = {"STATE NAME":'state', "Yes%": "yes_percent"}, inplace = True) 
alcohol_df = pd.read_csv(alcohol_file_name)
alcohol_df = alcohol_df.filter(["state","factor","yes_percent"])
income_df = pd.read_csv(income_file_name)
income_df = income_df.filter(["state","factor","yes_percent"])
poverty_df = pd.read_csv(poverty_file_name)
poverty_df = poverty_df.filter(["state","factor","yes_percent"])

#Writing the DataFrame into Sqlite tables
npao_df.to_sql(con=engine, index_label='id', name='npao', if_exists='replace')
depression_df.to_sql(con=engine, index_label='id', name='depression', if_exists='replace')
alcohol_df.to_sql(con=engine, index_label='id', name='alcohol', if_exists='replace')
income_df.to_sql(con=engine, index_label='id',name='income', if_exists='replace')
poverty_df.to_sql(con=engine, index_label='id', name='poverty', if_exists='replace')
con = engine.connect() 
#Filtering out data for Obesity from the NutritionPhysicalActivityObesity table
sql_query = "select LocationDesc as state,data_value as factor, yes_percent  from npao inner join depression on upper(npao.LocationDesc) = upper(depression.state) where npao.Total = 'Total' and QuestionID = 'Q036' and LocationDesc <> 'National' and YearStart = '2014'"
#Creating DataFrame for Obesity data
obesity_df = pd.read_sql(sql_query, con)
obesity_df.to_sql(con=engine, index_label='id', name='obesity', if_exists='replace')


In [4]:
print(pd.read_sql('select * from alcohol', con))

    id           state     factor  yes_percent
0    0         ALABAMA  10.945274    68.852459
1    1          ALASKA  13.793103    63.636364
2    2         ARIZONA  13.284133    66.666667
3    3        ARKANSAS  11.904762    68.421053
4    4      CALIFORNIA  15.719947    67.032967
5    5        COLORADO  16.888889    69.473684
6    6     CONNECTICUT  16.083916    69.230769
7    7        DELAWARE  20.000000    66.666667
8    8         FLORIDA  12.453532    65.745856
9    9         GEORGIA  10.723861    67.543860
10  10          HAWAII  14.545455    70.000000
11  11           IDAHO  15.517241    60.000000
12  12        ILLINOIS  17.283951    63.809524
13  13         INDIANA  15.040650    66.304348
14  14            IOWA  20.535714    64.102564
15  15          KANSAS  14.018692    64.864865
16  16        KENTUCKY  17.361111    72.727273
17  17       LOUISIANA  19.075145    67.796610
18  18           MAINE  16.000000    64.000000
19  19        MARYLAND  14.285714    69.148936
20  20   MASS

In [6]:
#Creating empty Dictionaries for individual data sets
depression = {}
obesity = {}
alcohol = {}
income = {}
poverty = {}

#Querying data from Database and saving it in variables
depression_result = con.execute("select * from depression")
obesity_result = con.execute("select * from obesity")
alcohol_result = con.execute("select * from alcohol")
income_result = con.execute("select * from income where STATE <> 'United States'")
poverty_result = con.execute("select * from poverty where STATE <> 'United States'")

#Appending data to individual dictionaries
for row in depression_result:
    depression[row[1].upper()] = round(row[2],2)
for row in obesity_result:
    obesity[row[1].upper()] = row[2]
for row in alcohol_result:
    alcohol[row[1].upper()] = round(row[2],2)   
for row in income_result:
    income[row[1].upper()] = row[2].replace(",","")  
for row in poverty_result:
    poverty[row[1].upper()] = row[2]  

print(income)
# Reading the State Co-ordinated GeoJSON file
with open('StateCoord_geojson.json', 'r') as geo_json:
    json_load_dep = json.load(geo_json)
    
# #Appending Depression data to the GeoJSON file
for k, v in depression.items():
    for i in range(len(json_load_dep['features'])):
        if json_load_dep['features'][i]['properties']['NAME'].upper() == k:
            json_load_dep['features'][i]['properties']['DEPRESSION'] = v            
            json_load_dep['features'][i]['properties']['FACTOR'] = 'Depression'

#Writing the Depression GeoJSON into a new file
with open('depression_geojson.json', 'w') as dep_geo_json:
    json.dump(json_load_dep, dep_geo_json)
    

# Reading the State Co-ordinated GeoJSON file
with open('StateCoord_geojson.json', 'r') as geo_json:
    json_load_alcohol = json.load(geo_json)
    
# #Appending Alcohol data to the GeoJSON file
for k, v in alcohol.items():
    for i in range(len(json_load_alcohol['features'])):
        if json_load_alcohol['features'][i]['properties']['NAME'].upper() == k:
            json_load_alcohol['features'][i]['properties']['ALCOHOL'] = v            
            json_load_alcohol['features'][i]['properties']['FACTOR'] = 'Alcohol'
            

#Writing the Alcohol GeoJSON into a new file
with open('alcohol_geojson.json', 'w') as alcohol_geo_json:
    json.dump(json_load_alcohol, alcohol_geo_json)

# Reading the State Co-ordinated GeoJSON file
with open('StateCoord_geojson.json', 'r') as geo_json:
    json_load_obesity = json.load(geo_json)
    
# #Appending obesity data to the GeoJSON file
for k, v in obesity.items():
    for i in range(len(json_load_obesity['features'])):
        if json_load_obesity['features'][i]['properties']['NAME'].upper() == k:
            json_load_obesity['features'][i]['properties']['OBESITY'] = v            
            json_load_obesity['features'][i]['properties']['FACTOR'] = 'Obesity'
            

#Writing the obesity GeoJSON into a new file
with open('obesity_geojson.json', 'w') as obesity_geo_json:
    json.dump(json_load_obesity, obesity_geo_json)
    

# Reading the State Co-ordinated GeoJSON file
with open('StateCoord_geojson.json', 'r') as geo_json:
    json_load_income = json.load(geo_json)
    
# #Appending income data to the GeoJSON file
for k, v in income.items():
    for i in range(len(json_load_income['features'])):
        if json_load_income['features'][i]['properties']['NAME'].upper() == k:
            json_load_income['features'][i]['properties']['INCOME'] = v            
            json_load_income['features'][i]['properties']['FACTOR'] = 'Income'
            

#Writing the income GeoJSON into a new file
with open('income_geojson.json', 'w') as income_geo_json:
    json.dump(json_load_income, income_geo_json)

# Reading the State Co-ordinated GeoJSON file
with open('StateCoord_geojson.json', 'r') as geo_json:
    json_load_poverty = json.load(geo_json)
    
# #Appending income data to the GeoJSON file
for k, v in poverty.items():
    for i in range(len(json_load_poverty['features'])):
        if json_load_poverty['features'][i]['properties']['NAME'].upper() == k:
            json_load_poverty['features'][i]['properties']['POVERTY'] = v            
            json_load_poverty['features'][i]['properties']['FACTOR'] = 'Poverty'
            

#Writing the income GeoJSON into a new file
with open('poverty_geojson.json', 'w') as poverty_geo_json:
    json.dump(json_load_poverty, poverty_geo_json)



{'ALABAMA': '48193', 'ALASKA': '74058', 'ARIZONA': '56508', 'ARKANSAS': '45916', 'CALIFORNIA': '71785', 'COLORADO': '69113', 'CONNECTICUT': '74428', 'DELAWARE': '64085', 'FLORIDA': '52582', 'GEORGIA': '56117', 'HAWAII': '77936', 'IDAHO': '52280', 'ILLINOIS': '63044', 'INDIANA': '54134', 'IOWA': '58706', 'KANSAS': '56382', 'KENTUCKY': '48332', 'LOUISIANA': '46283', 'MAINE': '55980', 'MARYLAND': '80711', 'MASSACHUSETTS': '77385', 'MICHIGAN': '54840', 'MINNESOTA': '68364', 'MISSISSIPPI': '43595', 'MISSOURI': '53506', 'MONTANA': '53262', 'NEBRASKA': '59913', 'NEVADA': '57932', 'NEW HAMPSHIRE': '73638', 'NEW JERSEY': '80106', 'NEW MEXICO': '47086', 'NEW YORK': '64783', 'NORTH CAROLINA': '52797', 'NORTH DAKOTA': '62400', 'OHIO': '54077', 'OKLAHOMA': '50051', 'OREGON': '60123', 'PENNSYLVANIA': '59165', 'RHODE ISLAND': '62923', 'SOUTH CAROLINA': '50675', 'SOUTH DAKOTA': '56871', 'TENNESSEE': '51319', 'TEXAS': '59195', 'UTAH': '68395', 'VERMONT': '58271', 'VIRGINIA': '71518', 'WASHINGTON': '709